# Rheostat with VirE2
In this notebook, I will outline a possible model that used in conjunction with Agrima VirE2 Model.<br>
7.12.2020<br>
Ankita Roychoudhury
----

In [1]:
# bioscrape
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pandas as pd

from scipy.integrate import simps
from numpy import trapz

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
bokeh.io.output_notebook()

# from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
# from biocrnpyler import Mixture
# from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
from biocrnpyler import *

Loading BokehJS ...

## Try to combine with subsbml
---

In [2]:
from subsbml import createNewSubsystem, createSubsystem

In [3]:
ss1 = createSubsystem('vire2_SBML_model_sec.xml', subsystemName = 'vire2')
ss2 = createSubsystem('ATP_rheostat.xml', subsystemName = 'rheostat')

# rename to have same things
ss1.renameSName('ATP', 'metabolite_atp')
ss1.renameSName('ADP', 'metabolite_adp')
ss1.renameSName('P', 'metabolite_pi')

<SBMLDocument>

In [4]:
atp_regen_subsystem = ss1 + ss2

In [5]:
ATP_id = atp_regen_subsystem.getSpeciesByName('metabolite_atp').getId()
VirE2_b_id = atp_regen_subsystem.getSpeciesByName('VirE2_b').getId()
VirE2_f_id = atp_regen_subsystem.getSpeciesByName('VirE2_f').getId()
atc_i_id = atp_regen_subsystem.getSpeciesByName('aTc_i').getId()




# SET RHEOSTAT INITIAL CONDITIONS
rheostat_30 = ['molecule_glucose', 'metabolite_atp', 'metabolite_nadp', 'metabolite_pi']
for spec in rheostat_30:
    atp_regen_subsystem.setSpeciesAmount(spec,30)
    if spec == 'metabolite_atp':
        atp_regen_subsystem.setSpeciesAmount(spec,30)


rheostat_e = ['enzyme_hex', 'enzyme_pgi', 'enzyme_pfk', 'enzyme_ald_tpi', 'enzyme_gapN',
             'enzyme_mGapDH', 'enzyme_pgk', 'enzyme_pgm', 'enzyme_eno', 'enzyme_pyk',
             'enzyme_alsS', 'enzyme_IlvC', 'enzyme_kivD', 'enzyme_yahk']

for spec_e in rheostat_e:
    atp_regen_subsystem.setSpeciesAmount(spec_e, 0.20)

# SET VIRE2 INITIAL CONDITIONS
atp_regen_subsystem.setSpeciesAmount('aTc_i', 15)
atp_regen_subsystem.setSpeciesAmount('dna_g', 1)
atp_regen_subsystem.setSpeciesAmount('tetR', 10)
#atp_regen_subsystem.setSpeciesAmount('ati', 15)







_ = atp_regen_subsystem.writeSBML('atp_regen_ss.xml')



In [6]:
# Simulate with Bioscrape and plot the result

#colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']


timepoints = np.linspace(0,200,100)
results, _ = atp_regen_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)


p1 = bokeh.plotting.figure(width = 600, height = 350,
                          title = 'VirE2 with Rheostat',
                          y_axis_label = 'Concentration (mM)',
                          x_axis_label = 'Time (secs)')

p1.line(timepoints,results[VirE2_b_id], color = colors[0], line_width = 3, legend_label = 'VirE2_b' )
#p1.line(timepoints,results[atc_i_id], color = colors[1], line_width = 3, legend_label = 'atc_i' )


p1.line(timepoints,results[VirE2_f_id], color = colors[2], line_width = 3, legend_label = 'VirE2_f' )
p1.line(timepoints,results[ATP_id], color = colors[3], line_width = 3, legend_label = 'atp' )


bokeh.io.show(p1)



# plt.figure(figsize=(12,6))


# plt.plot(timepoints, results[ATP_id], linewidth = 3, label = 'ATP',  )
# plt.plot(timepoints, results[VirE2_b_id], color = 'red', lw = 4, label='VirE2_b')
# plt.plot(timepoints, results[atc_i_id], color = 'green', lw = 4, label='atc_i')
# plt.plot(timepoints, results[VirE2_f_id], color = 'cyan', lw = 2, label='VirE2_f')


# plt.title('VirE2 with ATP Rheostat')
# plt.ylabel('[X]')
# plt.xlabel('Time')
# plt.legend()
# plt.show()



## Attach Agrima's Original Model Code Here 
From notebook VirE2 expression.ipynb<br>
8.2.2020
----


In [7]:
from biocrnpyler import *
import numpy as np
import pylab as plt

VirE2_f = Species ("VirE2_f") #, material_type = "protein") #free
repressor = Species("tetR") #, material_type = "protein")
dna_g= Species ("dna_g")
rna_g= Species ("rna_g")
ATP= Species ("ATP")
ADP= Species ("ADP")
atc_i = Species("aTc_i") # material_type = "ligand") #atc_i is the internal atc. 
#It binds to the repressor  (tetR) and frees the DNA to perform transcription
VirE2_b = Species ("VirE2_b") #bound to membran
p = Species("P")
d_r= ComplexSpecies([dna_g,repressor])
d_r_atc= ComplexSpecies([dna_g,repressor,atc_i])

#Create a ChemicalComplex to model ligand-repressor bindning
inactive_repressor = ChemicalComplex([repressor, atc_i])

species=[dna_g, repressor, rna_g, VirE2_f, ATP,ADP, atc_i, inactive_repressor.get_species(),VirE2_b, d_r, d_r_atc, p]

# rex1=Reaction([dna_g, repressor], [d_r], propensity_type = "massaction", k= 1000) #1e+11 #1
# rex2=Reaction([d_r,atc_i ], [d_r_atc], propensity_type = "massaction", k= 1000 ) # 6.5e+7 #10
# rex3=Reaction([d_r_atc], [dna_g, inactive_repressor.get_species()], propensity_type = "massaction", k= 10000, k_rev=10 ) #1000,10
# rex4=Reaction([dna_g], [dna_g, rna_g], propensity_type = "proportionalhillnegative", propensity_params = {"k":.0075, "n":1, "K":20, "s1":repressor, "d":dna_g})  #{"k":18.75, "n":2, "K":20, "s1":repressor, "d":dna_g}
# rex5=Reaction([dna_g], [dna_g, rna_g], propensity_type = "massaction", k= .001) 
# rex6=Reaction([rna_g, ATP], [rna_g, ADP, VirE2_f,P], propensity_type = "proportionalhillpositive", propensity_params = {"k":.25, "n":1, "K":1, "s1":ATP, "d":rna_g}) 
# rex7=Reaction([VirE2_f], [VirE2_b], propensity_type = "massaction", k= .1)
# rex8=Reaction([rna_g], [], propensity_type = "massaction", k= .01) 
# rex9=Reaction([VirE2_f], [], propensity_type = "massaction", k= 1e-4)


rex1=Reaction([dna_g, repressor], [d_r], propensity_type = "massaction",k= 1e-3, k_rev= 1e-5  ) #1e+11 #1
rex2=Reaction([d_r,atc_i ], [d_r_atc], propensity_type = "massaction", k= 6.5e-2, k_rev= 3e-4) # 6.5e+7 #10
rex3=Reaction([d_r_atc], [dna_g, inactive_repressor.get_species()], propensity_type = "massaction", k= 100, k_rev=10) #1000,10
rex4=Reaction([dna_g], [dna_g, rna_g], propensity_type = "proportionalhillnegative", propensity_params = {"k":0.01875, "n":1, "K":20, "s1":repressor, "d":dna_g})  #{"k":18.75, "n":2, "K":20, "s1":repressor, "d":dna_g}
rex5=Reaction([dna_g], [dna_g, rna_g], propensity_type = "massaction", k= .001) 
rex6=Reaction([rna_g, ATP], [rna_g, ADP, VirE2_f,p], propensity_type = "proportionalhillpositive", propensity_params = {"k":0.6, "n":1, "K":1, "s1":ATP, "d":rna_g}) 
rex7=Reaction([VirE2_f], [VirE2_b], propensity_type = "massaction", k= 1)
rex8=Reaction([rna_g], [], propensity_type = "massaction", k= .001) 
rex9=Reaction([VirE2_f], [], propensity_type = "massaction", k= 1e-5)

#rex4=Reaction([repressor, atc_i], [inactive_repressor.get_species()], propensity_type = "massaction", k= 100, k_rev=10) 


r=[rex1, rex2, rex3, rex4, rex5, rex6, rex7, rex8, rex9]


CRN1 = ChemicalReactionNetwork(species = species, reactions = r)


_ = CRN1.write_sbml_file('vire2_SBML_model_sec.xml')

print(CRN1.pretty_print())



Species (12) = {0. dna_g, 1. tetR, 2. rna_g, 3. VirE2_f, 4. ATP, 5. ADP, 6. aTc_i, 7. complex[aTc_i:tetR], 8. VirE2_b, 9. complex[dna_g:tetR], 10. complex[aTc_i:dna_g:tetR], 11. P}
Reactions (9) = [
0. dna_g + tetR <--> complex[dna_g:tetR]        
        massaction: k_f(dna_g,tetR)=0.001*dna_g*tetR
        k_r(complex[dna_g:tetR])=1e-05*complex[dna_g:tetR]
1. complex[dna_g:tetR] + aTc_i <--> complex[aTc_i:dna_g:tetR]        
        massaction: k_f(complex[dna_g:tetR],aTc_i)=0.065*complex[dna_g:tetR]*aTc_i
        k_r(complex[aTc_i:dna_g:tetR])=0.0003*complex[aTc_i:dna_g:tetR]
2. complex[aTc_i:dna_g:tetR] <--> dna_g + complex[aTc_i:tetR]        
        massaction: k_f(complex[aTc_i:dna_g:tetR])=100*complex[aTc_i:dna_g:tetR]
        k_r(dna_g,complex[aTc_i:tetR])=10*dna_g*complex[aTc_i:tetR]
3. dna_g --> dna_g + rna_g        
        proportionalhillnegative: k(tetR, dna_g)=0.01875*dna_g/(20+tetR^1)
4. dna_g --> dna_g + rna_g        
        massaction: k_f(dna_g)=0.001*dna_g
5. rna_g

In [8]:

 #initial condition
#x0_dict = {str(atc_i): 15, str(dna_g): 1, str(repressor):10} 
x0_dict = {str(atc_i): 15, str(dna_g): 1, str(repressor):10, str(ATP):30} 


time = np.linspace(0, 200, 1000)

R = CRN1.simulate_with_bioscrape(timepoints = time, initial_condition_dict = x0_dict)

p1 = bokeh.plotting.figure(width = 600, height = 350,
                          title = 'Only VirE2',
                          y_axis_label = 'Concentration (mM)',
                          x_axis_label = 'Time (secs)')
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']
p1.line(time,R[str(VirE2_b)], color = colors[0], line_width = 3, legend_label = 'VirE2_b' )
#p1.line(time,R[str(atc_i)], color = colors[1], line_width = 3, legend_label = 'atc_i' )


p1.line(time,R[str(VirE2_f)], color = colors[2], line_width = 3, legend_label = 'VirE2_f' )

p1.line(time,R[str(ATP)], color = colors[3], line_width = 3, legend_label = 'atp' )


bokeh.io.show(p1)





In [9]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,biocrnpyler,bioscrape

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
jupyterlab 1.2.6
biocrnpyler unknown
bioscrape 1.0.0
